# NanoGPT

In [46]:
# read it in to inspect it
with open('tiny_shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [47]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [48]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [49]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [50]:
# create a mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [51]:
import torch 

In [52]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [53]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

len(train_data), len(val_data)

(1003854, 111540)

In [54]:
block_size = 8 # context length
train_data[:block_size+1] 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [55]:
# In a chunk of 9 characters, there are 8 individual training examples that get sent to the network.
# This also helps the network to see different context sizes, from 1 to block_size. This allows the network to learn how to predict in multiple context lengths.
# Let's take a look. 
X = train_data[:block_size]
Y = train_data[1:block_size + 1]
for t in range(block_size):
    context = X[:t+1]
    target = Y[t]
    print(f"When input is {context}, the target is: {target}")

When input is tensor([18]), the target is: 47
When input is tensor([18, 47]), the target is: 56
When input is tensor([18, 47, 56]), the target is: 57
When input is tensor([18, 47, 56, 57]), the target is: 58
When input is tensor([18, 47, 56, 57, 58]), the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [56]:
# For efficiency purposes, we'll add an extra "batch" dimension to take advantage of GPU parallelism
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) -  block_size, (batch_size, )) # 4 numbers randomly generated between 0 and len(data) -  block_size
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("Inputs:")
print(xb.shape)
print(xb)
print("Targets:")
print(yb.shape)
print(yb)
print("---")

# The total number of examples here would be 8 x 4 = 32 independent examples, packed into a single batch x with targets y, which will all be simultaneously processed by the transformers
for b in range(batch_size): # batch dim
    for t in range(block_size): # time dim (context length)
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context}, the target is: {target}")


Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
When input is tensor([24]), the target is: 43
When input is tensor([24, 43]), the target is: 58
When input is tensor([24, 43, 58]), the target is: 5
When input is tensor([24, 43, 58,  5]), the target is: 57
When input is tensor([24, 43, 58,  5, 57]), the target is: 1
When input is tensor([24, 43, 58,  5, 57,  1]), the target is: 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]), the target is: 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is: 39
When input is tensor([44]), the target is: 53
When input is tensor([44, 53]), the target is: 56
When input is tensor([44, 53,

## Simplest version: Bigram Language Model

In [57]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers where B = Batch size (4) and T = Time/Context Size (8)
        logits = self.token_embedding_table(idx) # (B, T, C) where C is channel/embedding dimension (vocab size, 65)
        
        if targets is None:
            loss = None
        else:
            # reshape the logits tensor so that it is (B*T, C) because PyTorch expects the channel dimension to be the second dimension
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
        
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        # Generate (B, T + 1), (B, T + 2), ... (B, T + max_new_tokens)
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # Becomes (B, C)
            
            # convert to probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

# First result is random if it's not trained
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)

lfJeukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs
lZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH


In [58]:
# Create an optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [59]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch("train")
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    

print(loss.item())

2.362440586090088


In [60]:
# This is a very simple model because we're only looking at the last character to see what the next character will be!
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


M:
IUSh t,
F th he d ke alved.
Thupld, cipbll t
I: ir w, l me sie hend lor ito'l an e

I:
Gochosen ea ar btamandd halind
Aust, plt t wadyotl
I bel qunganonoth he m he de avellis k'l, tond soran:

WI he toust are bot g e n t s d je hid t his IAces I my ig t
Ril'swoll e pupat inouleacends-athiqu heame


## Self-attention: The mathematical trick

In [61]:
# Starting with a toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

We want tokens to talk to each other (couple-them)

For example: The token at the 5th location should communicate to tokens in previous locations, and NOT from tokens in "future" locations.

Information should only flow in one direction.

How to do this? I'd like to average the features on tokens 1-5, which would represent a summary of the fifth token in the context of its history (tokens 1-4).

A downside is that we'd lose a lot of information by compressing all the tokens like that but we'll worry about that later.

For now, 

```
for every batch:
    for every t-th token in that sequence:
        calculate the average of all the tokens up to the current one.
```




In [62]:
# We want: x[b, t] = mean_{i <= t} x[b_i]
xbow = torch.zeros((B, T, C)) # bow = bag of words, term to mean "averaging". I wish we would call this x_mean haha
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [73]:
# but there's a more efficient way of doing this! Cool matrix multiplication trick :) 
# let's see it in an example
torch.manual_seed(42)

# This generates a "triangular" matrix where the lower triangle (including the diagonal) contains ones and the upper one contains zeros
# (Specifically, torch.tril zeros out the upper triangular part of the matrix)
# Why is this useful? Consider the case where you multiply/dot product this matrix A (of shape m x n) by another matrix B (of shape n x k)
# Every column in the resulting matrix will contain:
# C[i, j] = A[i] @ B[j]
# Which translated to our problem, it basically means we have added up all the values of column j of matrix B up to current row i (which would be "t")
a = torch.tril(torch.ones(3, 3))
print(f"{a=}")

# and then we can divide it by the number of rows we're adding up (normalizing so all the rows add up to 1) :O :O genius
# somehow this results in an average. I still need to internalize it but it's cool.
# intuition: averaging ~= normalization? NO! 
# when we divide a sum of numbers by a constant C, that is the same as multiplying each of those numbers by 1/C!!!!!! LMAOOOO and each row in this triangular matrix contains the 1/C "weight" to multiply the list of numbers by. BAM!!
# -> how much is each value contributing to the final average value... the "weight" of each number in matrix B is defined in matrix A!
a = a / torch.sum(a, dim=1, keepdim=True)
print(f"{a=}")
print(f"---")

b = torch.randint(0, 10, (3, 2)).float()
print(f"{b=}")
print(f"---")

c = a @ b
print(f"{c=}")

a=tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [75]:
# Ok, then let's vectorize our operation: x[b, t] = mean_{i <= t} x[b_i]
# REMEMBER, the goal here is that a token in the t-th position only gets information from all the tokens preceding it! 
weights = torch.tril(torch.ones(T, T))
weights = weights / weights.sum(1, keepdim=True)
print(f"{weights=}")


# (T, T) @ (B, T, C) -- add a batch dim --> (B, T, T) @ (B, T, C) = (B, T, C)
xbow2 = weights @ x 
torch.allclose(xbow, xbow2)

weights=tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [79]:
# Lastly, another way of doing this would be... THE SOFTMAX!!!!!!!
tril = torch.tril(torch.ones(T, T))

# INTERPRETATION PT 1: Weights begin at 0, which represent interaction strength/affinity. How much of each token from the past do we want to aggregate/average up? 
#     At the beginning, all tokens can communicate with all tokens, but each of them has an "affinity" of zero to the current token
weights = torch.zeros((T, T))

# Make all the zeros in the upper triangular part of the matrix to be negative infinity because...
print(f"Tril: \n{tril}")
print(f"Weights before masking: \n{weights}")

# INTERPRETATION PT 2: This is limiting communication to tokens from the past by making all tokens from the future of the current token to have an affinity of negative infinity (So we WON'T aggregate ANYTHING from those tokens).
weights = weights.masked_fill(tril == 0, float("-inf"))
print(f"Weights after masking: \n{weights}")

# ... softmax will convert those to zero, and then it will normalize each row in the lower triangular part of the matrix :O :O :O 
# INTERPRETATION PT 3: This sets the amount of "affinity" that each token from the past will have when they get aggregated/averaged up, and for now each past token has an equal amount of "affinity" to the current token, which is 1/T (where T is the index of the current token)
weights = F.softmax(weights, dim=-1)
print(f"Weights after softmax: \n{weights}")

# INTERPRETATION PT 4: Here is where we aggregate the tokens with the affinity matrix/weights, which will end up being a reflection of how "interesting" each token finds each other (for now, equally interesting)
xbow3 = weights @ x
torch.allclose(xbow, xbow3)


# CONCLUSION: You can do weighted aggregation of your past elements by doing matrix multiplication of a lower-triangular fashion, where the elements of the lower triangular part of the matrix tell you how much of each element "fuses" into the current token/position
# TRIPLE BAM!

Tril: 
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
Weights before masking: 
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
Weights after masking: 
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0

True